# Backtesting Analysis Notebook

This notebook replicates the functionality of backtest_runner.py in an interactive format for faster iteration and testing.

In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import os
import vectorbt as vbt

# Filter warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)  # For div by zero, etc
warnings.filterwarnings('ignore', category=FutureWarning)   # For pandas operations
warnings.filterwarnings('ignore', category=UserWarning)     # For matplotlib
np.seterr(all='ignore')  # Ignore numpy warnings
pd.options.mode.chained_assignment = None  # Ignore pandas chained assignment warnings

# Add parent directory to path to import strategy modules
import sys
sys.path.append('..')

# Add parent directory to path to import strategy modules
import sys
sys.path.append('..')

from example_scripts.strategy_0_buy_and_hold import BuyAndHoldStrategy  # Add this line
from example_scripts.strategy_1_momentum import DualMomentumStrategy
from example_scripts.strategy_2_regime import MacroRegimeStrategy
from example_scripts.strategy_3_mean_reversion import MeanReversionStrategy
from example_scripts.strategy_4_multi_factor import MultiFactorStrategy
from example_scripts.strategy_5_volatility_regime import VolatilityRegimeStrategy
from example_scripts.strategy_6_adaptive_trend import AdaptiveTrendStrategy
from example_scripts.strategy_8_combined import CombinedStrategy
from example_scripts.strategy_analysis import print_strategy_analysis

## Data Loading and Overview

In [2]:
def load_data() -> pd.DataFrame:
    """Load and preprocess data"""
    data_path = Path('..') / 'raw_data' / 'df.csv'
    df = pd.read_csv(data_path, index_col=0, parse_dates=True)
    return df

# Load data
df = load_data()

# Print overview
print("Data Overview:")
print("=============")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Number of rows: {len(df)}")
print("\nFirst few rows:")
df.head()

Data Overview:
Date range: 2002-10-31 00:00:00 to 2024-12-27 00:00:00
Number of rows: 5562

First few rows:


,cad_ig_er_ytd_index,us_hy_er_ytd_index,cad_oas,us_hy_oas,us_ig_oas,tsx,vix,us_3m_10y,us_growth_surprises,us_inflation_surprises,us_lei_yoy,us_hard_data_surprises,us_equity_revisions,us_economic_regime
Date,,,,,,,,,,,,,,
2002-10-31,99.769150,81.017657,86.896937,1001.548411,253.069203,6248.79,31.14,353.472,0.09781,-0.01729,3.5,0.2097,-0.29,0.85
2002-11-01,100.005722,81.426533,96.391472,1007.211190,246.375560,6321.40,29.30,361.891,0.09228,-0.05664,3.5,0.1596,-0.36,0.85
2002-11-04,100.018898,81.978501,97.038166,998.114405,238.348497,6432.65,30.82,362.832,0.11054,-0.05636,3.5,0.1661,-0.36,0.85
2002-11-05,100.147307,82.210354,98.000565,993.579009,236.329661,6399.64,31.23,366.201,0.10998,-0.05608,3.5,0.1661,-0.36,0.85
2002-11-06,100.159541,82.212603,89.366061,994.224980,232.232518,6437.91,30.73,383.198,0.10943,-0.05580,3.5,0.1661,-0.36,0.85


## Strategy Setup and Backtesting

In [3]:
def print_metrics_table(results_df: pd.DataFrame):
    """Print a formatted table of strategy metrics"""
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    
    # Group metrics by category
    metric_groups = {
        'Returns': ['total_return', 'annual_return', 'daily_vol', 'downside_vol', 'max_drawdown'],
        'Risk-Adjusted': ['sharpe_ratio', 'sortino_ratio', 'calmar_ratio'],
        'Trading': ['total_trades', 'win_rate', 'avg_win', 'avg_loss', 'profit_factor', 'market_coverage', 'avg_hold_time'],
        'Regime': ['high_vol_return', 'low_vol_return', 'avg_vix', 'avg_regime'],
        'Risk': ['return_skew', 'return_kurtosis', 'return_autocorr', 'time_in_drawdown', 'recovery_ratio'],
        'Streaks': ['max_consecutive_wins', 'max_consecutive_losses']
    }
    
    for group_name, metrics in metric_groups.items():
        print(f"\n{group_name} Metrics:")
        print("-" * (len(group_name) + 8))
        
        # Select metrics for this group
        group_df = results_df[metrics].copy()
        
        # Format percentages
        pct_metrics = ['total_return', 'annual_return', 'daily_vol', 'downside_vol', 
                      'max_drawdown', 'win_rate', 'market_coverage', 'high_vol_return', 
                      'low_vol_return', 'time_in_drawdown']
        
        # Format each column
        for col in group_df.columns:
            if col in pct_metrics:
                group_df[col] = group_df[col].map('{:.1%}'.format)
            else:
                group_df[col] = group_df[col].map('{:.2f}'.format)
        
        display(group_df)

In [4]:
# Create strategy instances
strategies = [
    BuyAndHoldStrategy(df),  # Now it will be defined
    DualMomentumStrategy(df),
    MacroRegimeStrategy(df),
    MeanReversionStrategy(df),
    MultiFactorStrategy(df),
    VolatilityRegimeStrategy(df),
    AdaptiveTrendStrategy(df),
    CombinedStrategy(df)
]

# Run backtests and collect results
results = {}
for strategy in strategies:
    strategy_name = strategy.__class__.__name__
    print(f"Running backtest for {strategy_name}...")
    metrics = strategy.backtest()
    results[strategy_name] = metrics

# Convert to DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index')

# Print detailed metrics
print_metrics_table(results_df)

Running backtest for BuyAndHoldStrategy...
Running backtest for DualMomentumStrategy...
Running backtest for MacroRegimeStrategy...
Running backtest for MeanReversionStrategy...

Mean Reversion Strategy Signal Analysis:
spread_signal_20: 108 days (1.9% of time)
spread_signal_60: 72 days (1.3% of time)
combined_spread_signals: 149 days (2.7% of time)
mean_reverting: 3955 days (71.1% of time)
vol_filter: 4244 days (76.3% of time)
trend_filter: 3937 days (70.8% of time)
Running backtest for MultiFactorStrategy...

Multi-Factor Strategy Analysis:
Average Value Score: 0.11
Average Momentum Score: -0.18
Average Volatility Score: -0.12
Average Macro Score: -0.22
Days in Market: 1211 (21.8% of time)
Risk-On Days: 2846 (51.2% of time)
Number of Trades: 1294
Running backtest for VolatilityRegimeStrategy...

Volatility Regime Strategy Analysis:
Low Volatility Regime: 3102 days (55.8% of time)
Average Trend Strength: 0.26
Average Correlation Score: 0.14
Running backtest for AdaptiveTrendStrategy..

,total_return,annual_return,daily_vol,downside_vol,max_drawdown
BuyAndHoldStrategy,34.7%,1.4%,1.5%,1.5%,-15.5%
DualMomentumStrategy,-0.1%,-0.0%,0.2%,0.8%,-1.6%
MacroRegimeStrategy,25.6%,1.0%,0.6%,0.8%,-0.8%
MeanReversionStrategy,0.2%,0.0%,0.1%,0.7%,-0.7%
MultiFactorStrategy,27.5%,1.1%,0.7%,1.1%,-3.0%
VolatilityRegimeStrategy,9.9%,0.4%,0.3%,0.7%,-1.4%
AdaptiveTrendStrategy,125.3%,3.7%,0.9%,0.8%,-0.7%
CombinedStrategy,47.6%,1.8%,0.5%,0.7%,-0.4%



Risk-Adjusted Metrics:
---------------------


,sharpe_ratio,sortino_ratio,calmar_ratio
BuyAndHoldStrategy,0.90,0.87,0.09
DualMomentumStrategy,-0.03,-0.01,-0.00
MacroRegimeStrategy,1.81,1.25,1.23
MeanReversionStrategy,0.07,0.01,0.02
MultiFactorStrategy,1.64,0.96,0.37
VolatilityRegimeStrategy,1.24,0.58,0.31
AdaptiveTrendStrategy,4.18,4.40,5.09
CombinedStrategy,3.23,2.52,3.99



Trading Metrics:
---------------


...

,high_vol_return,low_vol_return,avg_vix,avg_regime
BuyAndHoldStrategy,-3.8%,4.1%,19.16,0.83
DualMomentumStrategy,-0.3%,0.1%,16.65,0.80
MacroRegimeStrategy,-0.0%,1.6%,18.73,0.87
MeanReversionStrategy,-0.0%,0.0%,15.58,0.98
MultiFactorStrategy,0.3%,1.5%,17.55,0.79
VolatilityRegimeStrategy,-0.1%,0.7%,14.88,0.88
AdaptiveTrendStrategy,2.4%,4.4%,17.85,0.83
CombinedStrategy,0.7%,2.3%,15.88,0.89



Risk Metrics:
------------


,return_skew,return_kurtosis,return_autocorr,time_in_drawdown,recovery_ratio
BuyAndHoldStrategy,-2.67,50.72,0.25,83.4%,0.00
DualMomentumStrategy,-1.25,4.57,0.04,72.6%,-0.00
MacroRegimeStrategy,1.47,14.67,0.24,72.7%,0.00
MeanReversionStrategy,1.97,7.82,-0.09,78.1%,0.00
MultiFactorStrategy,0.60,16.79,0.20,74.8%,0.00
VolatilityRegimeStrategy,-0.08,8.17,0.15,66.5%,0.00
AdaptiveTrendStrategy,1.81,22.00,0.14,51.5%,0.02
CombinedStrategy,2.05,20.84,0.22,57.7%,0.00



Streaks Metrics:
---------------


,max_consecutive_wins,max_consecutive_losses
BuyAndHoldStrategy,34.00,19.00
DualMomentumStrategy,11.00,5.00
MacroRegimeStrategy,34.00,9.00
MeanReversionStrategy,2.00,4.00
MultiFactorStrategy,9.00,3.00
VolatilityRegimeStrategy,20.00,14.00
AdaptiveTrendStrategy,34.00,7.00
CombinedStrategy,22.00,7.00


## Strategy Analysis

In [5]:
# Print strategy analysis
print_strategy_analysis(df, strategies)


Mean Reversion Strategy Signal Analysis:
spread_signal_20: 108 days (1.9% of time)
spread_signal_60: 72 days (1.3% of time)
combined_spread_signals: 149 days (2.7% of time)
mean_reverting: 3955 days (71.1% of time)
vol_filter: 4244 days (76.3% of time)
trend_filter: 3937 days (70.8% of time)

Multi-Factor Strategy Analysis:
Average Value Score: 0.11
Average Momentum Score: -0.18
Average Volatility Score: -0.12
Average Macro Score: -0.22
Days in Market: 1211 (21.8% of time)
Risk-On Days: 2846 (51.2% of time)
Number of Trades: 1294

Volatility Regime Strategy Analysis:
Low Volatility Regime: 3102 days (55.8% of time)
Average Trend Strength: 0.26
Average Correlation Score: 0.14

Adaptive Trend Strategy Analysis:
Average Trend Strength: 0.62
Average Efficiency Ratio: 0.16
Average Lookback Period: 22 days

Adaptive Trend Strategy Analysis:
Average Trend Strength: 0.62
Average Efficiency Ratio: 0.16
Average Lookback Period: 22 days

ML Ensemble Strategy Analysis:
Total Trading Days: 1299
Av

## Save Results (Optional)

In [6]:
# Save results
results_path = Path('..') / 'results' / 'backtest_results.csv'
results_path.parent.mkdir(exist_ok=True)
results_df.to_csv(results_path)
print(f"Results saved to: {os.path.abspath(results_path)}")

Results saved to: c:\Users\Eddy\Documents\python_projects\windsurf_bloomberg\results\backtest_results.csv
